In [ ]:

import torch
from torch import nn
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
import math
import os.path
import pandas as pd
from sklearn.model_selection import train_test_split
from PIL import Image
from glob import glob
from pandas import DataFrame

import random
import numpy as np
import os


import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import copy


SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True
    print(torch.cuda.get_device_name(0))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def prRed(skk): print("\033[91m {}\033[00m" .format(skk))
def prGreen(skk): print("\033[92m {}\033[00m" .format(skk))


Tesla V100-SXM2-16GB
---------SFLV1 ResNet18 on Covid-19----------


In [ ]:
# No. of users
num_users = 2
epochs = 50
frac = 1
lr = 0.0001
num_classes = 3
criterion = nn.CrossEntropyLoss()


In [ ]:

num_users = 2
epochs = 200
frac = 1        
lr = 0.0001




In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:

from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np

data_path = '/content/drive/My Drive/case4'

train_dataset = datasets.ImageFolder(
    root=data_path + '/train1',
    transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(), 
    transforms.RandomRotation(30),  
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2), 
    transforms.RandomGrayscale(p=0.1),  
    transforms.RandomPerspective(),  
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])
)


test_dataset = datasets.ImageFolder(
    root=data_path + '/test1',
    transform=transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])
)


batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [ ]:

class_counts = {}
for class_idx, class_name in enumerate(train_dataset.classes):
    class_counts[class_name] = len([label for _, label in train_dataset.samples if label == class_idx])

print("Number of samples under each class:")
for class_name, count in class_counts.items():
    print(f"{class_name}: {count}")

class_counts = {}
for class_idx, class_name in enumerate(test_dataset.classes):
    class_counts[class_name] = len([label for _, label in test_dataset.samples if label == class_idx])

print("Number of samples under each class:")
for class_name, count in class_counts.items():
    print(f"{class_name}: {count}")


Number of samples under each class:
MildDemented: 386
ModerateDemented: 282
NonDemented: 640
Number of samples under each class:
MildDemented: 100
ModerateDemented: 59
NonDemented: 160


In [ ]:

#=====================================================================================================
#                           Client-side Model definition
#=====================================================================================================

class ResNet18_client_side(nn.Module):
    def __init__(self):
        super(ResNet18_client_side, self).__init__()
        self.layer1 = nn.Sequential (
                nn.Conv2d(3, 64, kernel_size = 7, stride = 2, padding = 3, bias = False),
                nn.BatchNorm2d(64),
                nn.ReLU (inplace = True),
                nn.MaxPool2d(kernel_size = 3, stride = 2, padding =1),
            )
        self.layer2 = nn.Sequential  (
                nn.Conv2d(64, 64, kernel_size = 3, stride = 1, padding = 1, bias = False),
                nn.BatchNorm2d(64),
                nn.ReLU (inplace = True),
                nn.Conv2d(64, 64, kernel_size = 3, stride = 1, padding = 1),
                nn.BatchNorm2d(64),
            )

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()


    def forward(self, x):
        resudial1 = F.relu(self.layer1(x))
        out1 = self.layer2(resudial1)
        out1 = out1 + resudial1
        resudial2 = F.relu(out1)
        return resudial2




net_glob_client = ResNet18_client_side()
if torch.cuda.device_count() > 1:
    print("We use",torch.cuda.device_count(), "GPUs")
    net_glob_client = nn.DataParallel(net_glob_client)

net_glob_client.to(device)
print(net_glob_client)


ResNet18_client_side(
  (layer1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)


In [ ]:

#%%%
#=====================================================================================================
#                           Server-side Model definition
#=====================================================================================================


num_classes =  3


class Baseblock(nn.Module):
    expansion = 1
    def __init__(self, input_planes, planes, stride=1, dim_change=None):
        super(Baseblock, self).__init__()
        self.conv1 = nn.Conv2d(input_planes, planes, stride=stride, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, stride=1, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(planes)
        self.dim_change = dim_change

    def forward(self, x):
        res = x
        output = F.relu(self.bn1(self.conv1(x)))
        output = self.bn2(self.conv2(output))

        if self.dim_change is not None:
            res = self.dim_change(res)

        output += res
        output = F.relu(output)

        return output

class ResNet18_server_side(nn.Module):
    def __init__(self, block, num_layers, num_classes):
        super(ResNet18_server_side, self).__init__()
        self.input_planes = 64

        # Modify the first layer to accept the correct number of input channels
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
        )

        self.layer4 = self._layer(block, 128, num_layers[0], stride=2)
        self.layer5 = self._layer(block, 256, num_layers[1], stride=2)
        self.layer6 = self._layer(block, 512, num_layers[2], stride=2)
        self.averagePool = nn.AvgPool2d(kernel_size=7, stride=1)
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        # Initialize weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _layer(self, block, planes, num_layers, stride=2):
        dim_change = None
        if stride != 1 or planes != self.input_planes * block.expansion:
            dim_change = nn.Sequential(
                nn.Conv2d(self.input_planes, planes * block.expansion, kernel_size=1, stride=stride),
                nn.BatchNorm2d(planes * block.expansion)
            )
        netLayers = [block(self.input_planes, planes, stride=stride, dim_change=dim_change)]
        self.input_planes = planes * block.expansion
        for _ in range(1, num_layers):
            netLayers.append(block(self.input_planes, planes))
            self.input_planes = planes * block.expansion

        return nn.Sequential(*netLayers)

    def forward(self, x):
        out2 = self.layer3(x)
        out2 = out2 + x  # adding the residual inputs -- downsampling not required in this layer
        x3 = F.relu(out2)

        x4 = self.layer4(x3)
        x5 = self.layer5(x4)
        x6 = self.layer6(x5)

        x7 = F.avg_pool2d(x6, 7)
        x8 = x7.view(x7.size(0), -1)
        y_hat = self.fc(x8)

        return y_hat

# Instantiate the server-side model
net_glob_server = ResNet18_server_side(Baseblock, [2, 2, 2], num_classes)


if torch.cuda.device_count() > 1:
    print("We use", torch.cuda.device_count(), "GPUs")
    net_glob_server = nn.DataParallel(net_glob_server)

net_glob_server.to(device)

# Display the model architecture
print(net_glob_server)


ResNet18_server_side(
  (layer3): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (layer4): Sequential(
    (0): Baseblock(
      (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (dim_change): Sequential(
        (0): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2))
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )

In [ ]:

# ====================================================================================
# For Server Side Loss and Accuracy
loss_train_collect = []
acc_train_collect = []
loss_test_collect = []
acc_test_collect = []
batch_acc_train = []
batch_loss_train = []
batch_acc_test = []
batch_loss_test = []

num_classes= 3 
criterion = nn.CrossEntropyLoss()

count1 = 0
count2 = 0



In [ ]:
#%%%
# ====================================================================================================
#                               Server Side Program
# ====================================================================================================
num_classes=3
criterion = nn.CrossEntropyLoss()

# Federated averaging: FedAvg
def FedAvg(w):
    w_avg = copy.deepcopy(w[0])
    for k in w_avg.keys():
        for i in range(1, len(w)):
            w_avg[k] += w[i][k]
        w_avg[k] = torch.div(w_avg[k], len(w))
    return w_avg


def calculate_accuracy(fx, y):
    preds = fx.max(1, keepdim=True)[1]
    correct = preds.eq(y.view_as(preds)).sum()
    acc = 100.00 * correct.float() / preds.shape[0]
    return acc

def calculate_precision(fx, y):
    preds = fx.max(1, keepdim=True)[1]

    true_positives = (preds.eq(y.view_as(preds)) & (y == 1)).sum().item()
    false_positives = (preds.ne(y.view_as(preds)) & (y == 0)).sum().item()

    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) != 0 else 0

    return precision

def calculate_recall(fx, y):
    preds = fx.max(1, keepdim=True)[1]

    true_positives = (preds.eq(y.view_as(preds)) & (y == 1)).sum().item()
    false_negatives = (preds.ne(y.view_as(preds)) & (y == 1)).sum().item()

    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) != 0 else 0

    return recall


acc_avg_all_user_train = 0
loss_avg_all_user_train = 0
loss_train_collect_user = []
acc_train_collect_user = []
loss_test_collect_user = []
acc_test_collect_user = []

w_glob_server = net_glob_server.state_dict()
w_locals_server = []

# client idx collector
idx_collect = []
l_epoch_check = False
fed_check = False

net_model_server = [ResNet18_server_side(Baseblock, [2, 2, 2, 2], num_classes) for i in range(num_users)]

net_server = copy.deepcopy(net_model_server[0]).to(device)
optimizer_server = torch.optim.Adam(net_server.parameters(), lr=lr)



In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import f1_score, roc_auc_score, roc_curve, auc



In [ ]:

# Server-side function associated with Training
def train_server(fx_client, y, l_epoch_count, l_epoch, idx, len_batch):
    global net_model_server, criterion, optimizer_server, device, batch_acc_train, batch_loss_train, l_epoch_check, fed_check
    global loss_train_collect, acc_train_collect, count1, acc_avg_all_user_train, loss_avg_all_user_train, idx_collect, w_locals_server, w_glob_server, net_server
    global loss_train_collect_user, acc_train_collect_user, lr

    net_server = copy.deepcopy(net_model_server[idx]).to(device)
    net_server.train()
    optimizer_server = torch.optim.Adam(net_server.parameters(), lr=lr)

    # Train and update
    optimizer_server.zero_grad()

    fx_client = fx_client.to(device)
    y = y.to(device)

    # Forward prop
    fx_server = net_server(fx_client)

    # Calculate loss
    loss = criterion(fx_server, y)
    # Calculate accuracy
    acc = calculate_accuracy(fx_server, y)
    # Calculate precision
    precision = calculate_precision(fx_server, y)
    # Calculate recall
    recall = calculate_recall(fx_server, y)
    # Backward prop
    loss.backward()
    dfx_client = fx_client.grad.clone().detach()
    optimizer_server.step()

    batch_loss_train.append(loss.item())
    batch_acc_train.append(acc.item())

    net_model_server[idx] = copy.deepcopy(net_server)

    count1 += 1
    if count1 == len_batch:
        acc_avg_train = sum(batch_acc_train)/len(batch_acc_train)  
        loss_avg_train = sum(batch_loss_train)/len(batch_loss_train)

        batch_acc_train = []
        batch_loss_train = []
        count1 = 0

        prRed('Client{} Train => Local Epoch: {} \tAcc: {:.3f} \tLoss: {:.4f} \tPrecision: {:.3f} \tRecall: {:.3f} '.format(idx, l_epoch_count, acc_avg_train, loss_avg_train, precision, recall))

        w_server = net_server.state_dict()

        if l_epoch_count == l_epoch-1:

            l_epoch_check = True 
            w_locals_server.append(copy.deepcopy(w_server))


            acc_avg_train_all = acc_avg_train
            loss_avg_train_all = loss_avg_train

            loss_train_collect_user.append(loss_avg_train_all)
            acc_train_collect_user.append(acc_avg_train_all)

            if idx not in idx_collect:
                idx_collect.append(idx)

        if len(idx_collect) == num_users:
            fed_check = True  
            w_glob_server = FedAvg(w_locals_server)

            # Server-side global model update and distribute that model to all clients
            net_glob_server.load_state_dict(w_glob_server)
            net_model_server = [net_glob_server for i in range(num_users)]

            w_locals_server = []
            idx_collect = []

            acc_avg_all_user_train = sum(acc_train_collect_user)/len(acc_train_collect_user)
            loss_avg_all_user_train = sum(loss_train_collect_user)/len(loss_train_collect_user)

            loss_train_collect.append(loss_avg_all_user_train)
            acc_train_collect.append(acc_avg_all_user_train)

            acc_train_collect_user = []
            loss_train_collect_user = []

    # Send gradients to the client
    return dfx_client



In [ ]:

# Server-side functions associated with Testing
def evaluate_server(fx_client, y, idx, len_batch, ell):
    global net_model_server, criterion, batch_acc_test, batch_loss_test, check_fed, net_server, net_glob_server
    global loss_test_collect, acc_test_collect, count2, num_users, acc_avg_train_all, loss_avg_train_all, w_glob_server, l_epoch_check, fed_check
    global loss_test_collect_user, acc_test_collect_user, acc_avg_all_user_train, loss_avg_all_user_train
    batch_prec_test = []
    batch_recall_test = []
    net = copy.deepcopy(net_model_server[idx]).to(device)
    net.eval()

    with torch.no_grad():
        fx_client = fx_client.to(device)
        y = y.to(device)
        #---------forward prop-------------
        fx_server = net(fx_client)

        # Calculate loss
        loss = criterion(fx_server, y)
        # Calculate accuracy
        acc = calculate_accuracy(fx_server, y)

        y_true = y.cpu().numpy()
        y_pred = fx_server.cpu().max(1, keepdim=True)[1].numpy().flatten()
        y_score = F.softmax(fx_server, dim=1)
        precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
        recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)

        batch_loss_test.append(loss.item())
        batch_acc_test.append(acc.item())
        batch_prec_test.append(precision)
        batch_recall_test.append(recall)

        count2 += 1
        if count2 == len_batch:
            acc_avg_test = sum(batch_acc_test)/len(batch_acc_test)
            loss_avg_test = sum(batch_loss_test)/len(batch_loss_test)
             # Check if any batches were processed
            if len(batch_prec_test) > 0:
              prec_avg_test = sum(batch_prec_test) / len(batch_prec_test)
            else:
              prec_avg_test = 0  

            if len(batch_recall_test) > 0:
              recall_avg_test = sum(batch_recall_test) / len(batch_recall_test)
            else:
              recall_avg_test = 0  

            batch_acc_test = []
            batch_loss_test = []
            batch_prec_test = []
            batch_recall_test = []
            count2 = 0

            prGreen('Client{} Test =>                   \tAcc: {:.3f} \tLoss: {:.4f} \tPrecision: {:.3f} \tRecall: {:.3f} '.format(idx, acc_avg_test, loss_avg_test, prec_avg_test, recall_avg_test))

            if l_epoch_check:
                l_epoch_check = False

                acc_avg_test_all = acc_avg_test
                loss_avg_test_all = loss_avg_test

                loss_test_collect_user.append(loss_avg_test_all)
                acc_test_collect_user.append(acc_avg_test_all)

            if fed_check:
                fed_check = False
                print("------------------------------------------------")
                print("------ Federation process at Server-Side ------- ")
                print("------------------------------------------------")

                acc_avg_all_user = sum(acc_test_collect_user)/len(acc_test_collect_user)
                loss_avg_all_user = sum(loss_test_collect_user)/len(loss_test_collect_user)

                loss_test_collect.append(loss_avg_all_user)
                acc_test_collect.append(acc_avg_all_user)
                acc_test_collect_user = []
                loss_test_collect_user= []

                print("====================== SERVER V1==========================")
                print(' Train: Round {:3d}, Avg Accuracy {:.3f} | Avg Loss {:.3f}'.format(ell, acc_avg_all_user_train, loss_avg_all_user_train))
                print(' Test: Round {:3d}, Avg Accuracy {:.3f} | Avg Loss {:.3f}'.format(ell, acc_avg_all_user, loss_avg_all_user))
                print("==========================================================")

    return


In [ ]:
batch_prec_test = []
batch_recall_test = []

In [ ]:
from torch.optim.lr_scheduler import StepLR

def evaluate_accuracy(net, loader, device, return_conf_matrix=False):
    net.eval()
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for data in loader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = net(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = correct / total
    conf_matrix = confusion_matrix(all_labels, all_preds)

    if return_conf_matrix:
        return accuracy, conf_matrix
    else:
        return accuracy

#==============================================================================================================
#                                       Clients-side Program
#==============================================================================================================
class DatasetSplit(Dataset):
    def __init__(self, dataset, idxs):
        self.dataset = dataset
        self.idxs = list(idxs)

    def __len__(self):
        return len(self.idxs)

    def __getitem__(self, item):
        image, label = self.dataset[self.idxs[item]]
        return image, label

class Client(object):
    def __init__(self, net_client_model, idx, lr, device, train_loader=None, test_loader=None, idxs=None, idxs_test=None):
        self.idx = idx
        self.device = device
        self.lr = lr
        self.local_ep = 1
        self.ldr_train = train_loader
        self.ldr_test = test_loader

    def train(self, net):
        net.train()
        optimizer_client = torch.optim.Adam(net.parameters(), lr=self.lr)

        scheduler = StepLR(optimizer_client, step_size=30, gamma=0.1)

        for iter in range(self.local_ep):
            len_batch = len(self.ldr_train)
            for batch_idx, (images, labels) in enumerate(self.ldr_train):
                images, labels = images.to(self.device), labels.to(self.device)
                optimizer_client.zero_grad()
                #---------forward prop-------------
                fx = net(images)
                client_fx = fx.clone().detach().requires_grad_(True)

                # Sending activations to server and receiving gradients from server
                dfx = train_server(client_fx, labels, iter, self.local_ep, self.idx, len_batch)

                #--------backward prop -------------
                fx.backward(dfx)
                optimizer_client.step()

            # Step the learning rate scheduler
            scheduler.step()

        return net.state_dict()


    def evaluate(self, net, ell):
        net.eval()

        with torch.no_grad():
            len_batch = len(self.ldr_test)
            for batch_idx, (images, labels) in enumerate(self.ldr_test):
                images, labels = images.to(self.device), labels.to(self.device)
                #---------forward prop-------------
                fx = net(images)

                # Sending activations to server
                evaluate_server(fx, labels, self.idx, len_batch, ell)

        return

#=====================================================================================================
# dataset_iid() will create a dictionary to collect the indices of the data samples randomly for each client
def dataset_iid(dataset, num_users):

    num_items = int(len(dataset) / num_users)
    dict_users, all_idxs = {}, [i for i in range(len(dataset))]
    for i in range(num_users):
        dict_users[i] = set(np.random.choice(all_idxs, num_items, replace=False))
        all_idxs = list(set(all_idxs) - dict_users[i])
    return dict_users



In [ ]:
from pandas import DataFrame
import os
from google.colab import drive
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import f1_score, roc_auc_score, roc_curve, auc



import time

# Initialize a list to store the time taken for each epoch
epoch_times = []

dict_users = dataset_iid(train_dataset, num_users)
dict_users_test = dataset_iid(test_dataset, num_users)

# Training and Testing Loop
net_glob_client.train()

# Copy weights
w_glob_client = net_glob_client.state_dict()

# Federation takes place after certain local epochs in train() client-side
# This epoch is the global epoch, also known as rounds
for iter in range(epochs):
    start_time = time.time()  # Start time of the epoch
    m = max(int(frac * num_users), 1)
    idxs_users = np.random.choice(range(num_users), m, replace=False)
    w_locals_client = []

    for idx in idxs_users:
        local = Client(
            net_glob_client,
            idx,
            lr,
            device,
            train_loader=train_loader,
            test_loader=test_loader,
            idxs=dict_users[idx],
            idxs_test=dict_users_test[idx]
        )

        # Training
        w_client = local.train(net=copy.deepcopy(net_glob_client).to(device))
        w_locals_client.append(copy.deepcopy(w_client))

        # Testing
        local.evaluate(net=copy.deepcopy(net_glob_client).to(device), ell=iter)

    # After serving all clients for their local epochs
    # Federated Server: Federation process at Client-Side
    print("-----------------------------------------------------------")
    print("------ FedServer: Federation process at Client-Side ------- ")
    print("-----------------------------------------------------------")
    w_glob_client = FedAvg(w_locals_client)

    # Update client-side global model
    net_glob_client.load_state_dict(w_glob_client)
    # Calculate epoch time
    epoch_time = time.time() - start_time
    epoch_times.append(epoch_time)
    print(f"Epoch {iter + 1}/{epochs} - Time taken: {epoch_time:.2f} seconds")

# Training and Evaluation completed!
print("Training and Evaluation completed!")

# Save output data to .excel file (used for comparison plots)
round_process = [i for i in range(1, len(acc_train_collect) + 1)]
df = DataFrame({'round': round_process, 'acc_train': acc_train_collect, 'acc_test': acc_test_collect})
file_path = "/content/drive/MyDrive/Federated_Split_Learning_covid-19.xlsx"
df.to_excel(file_path, sheet_name="v1_test", index=False)

# Unmount Google Drive
drive.flush_and_unmount()
print(f"Excel file saved to: {file_path}")